In [ ]:
import pandas as pd 
import functools
import numpy as np
import warnings
import operator
from tqdm import tqdm
warnings.filterwarnings("ignore")

En este notebook se encuentran las funciones utilizadas para extraer características en texto como son las emociones y moralidad reflejadas
Se ha decidido utilizar estas características debido a la alta subjetividad de las tareas, para la extracción de éstas se ha utilizado un enfoque basado en léxicos donde se extraían las emociones o morales en función de si había palabras en el texto que coincidieran con un léxico de emociones (depechemood+) y de valores morales (moralstrength).
Para el léxico de emociones se ha hecho uso del léxico y se han creado funciones que recorrían el texto usando los valores (lemmas) de este léxico, para el uso del léxico de moralidad se ha hecho uso de una librería ya implementada para obtener los valores morales

# EMOCIONES

In [ ]:
def extract_emo_representation(words, emo_vocab=None, emotion_lex=None, n_emotions=None):
    """
    Extrae una representación de emociones a partir de una lista de palabras y devuelve también
    las palabras que contribuyen a esta representación.
    
    Args:
    words (list of str): Lista de palabras del texto.
    emo_vocab (set): Conjunto de palabras en el léxico emocional.
    emotion_lex (dict): Diccionario que mapea palabras a sus vectores de emoción.
    n_emotions (int): Número de emociones en los vectores de emoción.
    
    Returns:
    dict: Diccionario con 'emotion_vector' (representación promedio de emociones)
          y 'matched_words' (palabras usadas en el cálculo).
    """
    intersection = emo_vocab & set(words)
    matched_words = list(intersection) 
    v = np.zeros((len(intersection), n_emotions))
    
    for i, word in enumerate(intersection):
        v[i, :] = emotion_lex[word]
    
    # Calculamos el vector de emociones (usamos la media)
    emotion_vector = np.mean(v, axis=0) if len(intersection) > 0 else np.zeros(n_emotions)
    
    # Retornamos tanto el vector de emociones como las palabras coincidentes
    return {
        'emotion_vector': emotion_vector,
        'matched_words': matched_words
    }

    
def dictionary_emotion(text):
    """
    Convierte la lista puntuaciones de emociones en diccionario con emoción como clave
    Args:
    text (list): Lista de puntuaciones de emociones.
    
    Returns:
    dict: Diccionario con las emociones y sus respectivas emociones.
    """
    test_keys = ["fear", "amusement", "anger","annoyance","indifference","happiness","inspiration","sadness"]
    dictionary = dict(map(lambda i,j : (i,j) , test_keys,text))
    return dictionary


def top_n_emotions_names(emotion_scores, n=3):
    """
    Obtiene las n emociones con los puntajes más altos (solo nombres).
    
    Args:
    emotion_scores (dict): Diccionario que mapea etiquetas de emociones a sus respectivos puntajes.
    n (int): Número de emociones con los puntajes más altos a devolver.
    
    Returns:
    list: Lista de las n emociones con mayores puntajes.
    """
    sorted_emotions = sorted(emotion_scores.items(), key=lambda item: item[1], reverse=True)
    return [emotion for emotion, _ in sorted_emotions[:n]]


def get_max_emotion_name(emotion_scores):
    """
    Obtiene el nombre de la emoción con el puntaje más alto.
    
    Args:
    emotion_scores (dict): Diccionario que mapea etiquetas de emociones a sus respectivos puntajes.
    
    Returns:
    str: El nombre de la emoción con el puntaje más alto.
    """
    max_emotion = max(emotion_scores, key=emotion_scores.get)
    return max_emotion

## Léxico DepecheMood++ 

In [ ]:
# Leer léxico
#lexicon=pd.read_csv('DATASETS/DepecheMood_english_lemma_full.tsv',sep='\t',index_col=[0])
#lexicon.to_csv('DATASETS/DepecheMood_english_lemma_full.csv')

#Filtrar lexico para incluir solo filas más frecuentes 'freq' >= , 134278 valores del léxico descartados (23%), 41314 lemmas
lexicon=pd.read_csv('DepecheMood_english_lemma_full.csv',index_col=[0])
lexicon=lexicon[lexicon['freq'] >= 10] 

#Convertir léxico en diccionario por facilidad de uso
lexicon=lexicon.drop('freq',axis=1)
lexicon=lexicon.reset_index()
lexicon_dict = lexicon.set_index('index').T.to_dict('list')
#lexicon_dict
#lexicon.loc[200:250,:]

## Aplicar función de emociones a los datos aumentados

In [ ]:
df=pd.read_csv('data/train/dataset_en_train_augmented.csv')

In [ ]:
# Aplicamos las transformaciones de emociones
emo_vocab = set(lexicon_dict.keys())
n_emotions = 8  # Número de emociones

# Creamos listas vacías para almacenar los resultados
matched_words_list = []
top_3_emotions_list = []
max_emotion_list = []

# Actualizamos el bucle para usar el nuevo nombre de la función
for text in df['text']:
    # Extraemos el vector de emociones y las palabras coincidentes
    data = extract_emo_representation(text.split(' '), emo_vocab, lexicon_dict, n_emotions)
    emotion_vector = data['emotion_vector']
    matched_words = data['matched_words']
    
    # Convertimos el vector de emociones en un diccionario
    emotion_scores = dictionary_emotion(emotion_vector)
    
    # Calculamos los resultados necesarios
    top_3_emotions = top_n_emotions_names(emotion_scores, 3)  # Obtenemos solo los nombres
    max_emotion = get_max_emotion_name(emotion_scores)  # Obtenemos solo el nombre
    
    # Guardamos los resultados en las listas
    matched_words_list.append(matched_words)
    top_3_emotions_list.append(top_3_emotions)
    max_emotion_list.append(max_emotion)

# Añadimos las nuevas columnas al DataFrame
df['matched_words'] = matched_words_list
df['top_3_emotions'] = top_3_emotions_list
df['max_emotion'] = max_emotion_list


# VALORES MORALES

In [ ]:
def prompt_moralstrength2(text):
    """
    Procesa un texto para estimar valores morales usando una función auxiliar.
    
    Args:
        text (str): Texto de entrada.
        
    Returns:
        dict: Diccionario con valores morales y sus puntuaciones. Si no se encuentra ningún valor moral,
              devuelve un diccionario con 'moral': 'non-moral'.
    """
    result = estimate_morals(text, process=True)
    result=result.fillna(0)
    result=result.loc[0].to_dict()
    dict={}
    for elem in result:
        #if result[elem]!=0.0:
        dict[elem]=result[elem]
    if len(dict)==0:
        dict['moral']= 'non-moral'
    return dict


def prompt_moralstrength(df):
    """
    Procesa un dataframe para estimar valores morales en una columna de texto.
    
    Args:
        df (pd.DataFrame): DataFrame con una columna de texto ('text').
        
    Returns:
        pd.DataFrame: DataFrame con una nueva columna 'moralstrength' que contiene
                      diccionarios con valores morales estimados.
    """
    df['text_split']=df['text'].apply(lambda x: x.split('\n'))
    df['moralstrength'] = df['text_split'].apply(lambda x: prompt_moralstrength2(x))
    df.drop('text_split',axis=1,inplace=True)
    return df

def classify_values(values):
    """
    Clasifica los valores morales según su polaridad (positiva o negativa).
    
    Args:
        values (dict): Diccionario con valores morales y sus puntuaciones.
        
    Returns:
        dict: Diccionario con los valores clasificados en términos de polaridad.
    """
    classification = {}

    for moral, value in values.items():
        if moral == 'care':
            if value > 5:
                classification['care'] = value
            elif value < 5:
                classification['harm'] = value
        elif moral == 'fairness':
            if value > 5:
                classification['fairness'] = value
            elif value < 5:
                classification['cheating'] = value
        elif moral == 'loyalty':
            if value > 5:
                classification['loyalty'] = value
            elif value < 5:
                classification['betrayal'] = value
        elif moral == 'authority':
            if value > 5:
                classification['authority'] = value
            elif value < 5:
                classification['subversion'] = value
        elif moral == 'purity':
            if value > 5:
                classification['purity'] = value
            elif value < 5:
                classification['degradation'] = value

    return classification

def moralstrength_differences(valores):
    """
    Calcula la diferencia de cada valor moral respecto a un punto neutral (5).
    
    Args:
        valores (dict): Diccionario con valores morales y sus puntuaciones.
        
    Returns:
        dict: Diccionario con las diferencias calculadas, categorizadas en grande, moderada o pequeña.
    """
    clasificacion = {}
    for moral, valor in valores.items():
        if valor !=0:
            diferencia = abs(valor - 5)  # Calcula la diferencia respecto a 5
            if diferencia > 2.5:  # Diferencia grande
                clasificacion[moral] = diferencia
            elif diferencia > 1.5:  # Diferencia moderada
                clasificacion[moral] = diferencia
            else:  # Diferencia pequeña
                clasificacion[moral] = diferencia
       
    return clasificacion

def moralstrength_max_moral(values):
    """
    Encuentra el valor moral con la mayor magnitud de diferencia.
    
    Args:
        values (dict): Diccionario con valores morales y sus puntuaciones.
        
    Returns:
        str: Clave del valor moral con la mayor diferencia.
    """

    max_difference = -1
    max_key = None  
    for moral, value in values.items():
        difference = abs(value)
        if difference > max_difference:
            max_difference = difference
            max_key = moral
    return max_key



## Aplicar función de moralidad a los datos aumentados

In [ ]:
df = prompt_moralstrength(df)
df['moralstrength_base'] = df['moralstrength'].apply(lambda x: classify_values(x))
df['moralstrength'] = df['moralstrength_base'].apply(lambda x: moralstrength_differences(x))
df['prompt'] = df['moralstrength'].apply(lambda x: moralstrength_max_moral(x))

# GUARDAR

In [ ]:
#df.to_csv('data/train/dataset_en_train_completed.csv')